In looking at the flux results, I had a hunch that something was off. The flux was correct on my test flow fields, so I speculated the problem was in the flow approximations of the calcium. So I've been to digging into the idiosyncrasies of the flow calculations and I have found what I believe to an issue, hopefully THE issue.

---

Take frames 79 and 80 from the TSeries-01292015-1540_site3_0.75ISO_AL_VDNstd3 data set... (Frame 79 on left, frame 80 on right)

<img src="files/media/issues_with_flow/frames79and80.png" height="90%" width="90%">

We can clearly see from this raw data that there appears to be inward calcium movement in the top process. So I began to examine how accurately the optical flow algorithm captures this. From our discussions, the implementation that I WAS using went as follows

```python
# ...
data = loadData(dataPath) # Load the data
f0 = percentile(data, 10.0, axis=0) # Used for calculating relative fluorescence
relData = (data-f0)/f0 # Reletive fluorescence
blurData = gauss(relData, (1.2,0.75,0.75)) # Blurring stds are (time,y,x)
# ---- Then calculate the optical flow ----
# ...
prev = blurData[0]
for i,curr in enumerate(blurData[1:]):
    flow = optFlow(prev, curr, pyr_scale, levels, winSz, itrs, polyN, polyS, flg)
    xflow[i] = flow[:,:,0]
    yflow[i] = flow[:,:,1]
    prev = curr
```

So we see here that the flow is calculated using the blurred relative fluorescence. Below I show frames 79 and 80 from the the preblurred relative fluorescence, `relData`, and the post blurred relative fluorescence, `blurData`.

<img src="files/media/issues_with_flow/preblur_rc_frames79and80.png" height="92%" width="92%">
<img src="files/media/issues_with_flow/rcframes79and80.png" height="92%" width="92%">

I see a poor representation of the movement of calcium in the top process as compared with the original data. This can be seen from the flow approximations between these two frames. If you examine the flow more closely (I went back and forth between overlayed flows and images to see) you can see that the apparent inward flow in the top process is not aligned on top of the process and it is not at what visually appears to be the right angle. We compare the flow determined from both the preblurred relative calcium, and the blurred relative calcium

<img src="files/media/issues_with_flow/preblurred_rc_based_raw_flow.png" height="96%" width="96%">
<img src="files/media/issues_with_flow/rc_based_raw_flow.png" height="96%" width="96%">

So I decided to try the calculations again on the regular blurred fluorescence images. i.e. I simply removed the relative fluorescence calculation. The two blurred raw frames 79 and 80 are shown below...

<img src="files/media/issues_with_flow/rawframes79and80.png" height="92%" width="92%">

If I didn't save the image all goofy then it would be pretty clear that this better represents the calcium movement in the top process. (I am emailing you all of the figures so you can examine and compare them at your leisure) So, I was hoping that the flow calculations would be much better, however, they were not. Below is the flow approximation.

<img src="files/media/issues_with_flow/raw_flow.png" height="96%" width="96%">

After I carefully examined this it seemed to me that all of the alleged "correct" flows are in here, however there is also much more garbage. This is because the flow calculations look for displacement and ignore magnitude. So the seemingly strong flows around the periphery are allegedly the dynamics of the faint calcium fluorescence. So I remembered my previous idea of incorporating the original data set into the flux, and thought I could incorporate the original dataset into the flow calculations instead. So, I compared the results between scaling the flow by original blurred calcium intensities and via scaling it by the blurred relative calcium intensities. The idea here is that the "strong" flows around the periphery would be smothered by the low amplitude of the calcium underneath while the strong calcium movements associated with high calcium intensity would be accentuated. It turns out that this was the case...

Here is the scaled flow based on the relative fluorescence... The first is scaled by calcium intesnty. The second is scaled by relative calcium intensity

<img src="files/media/issues_with_flow/preblurred_rc_based_flow_scaled_by_calcium.png" height="96%" width="96%">
<img src="files/media/issues_with_flow/preblurred_rc_based_flow_scaled_by_relative_calcium.png" height="96%" width="96%">


And now the scaled flow based on the regular fluorescence... The first is scaled by calcium intesnty. The second is scaled by relative calcium intensity

<img src="files/media/issues_with_flow/flow_scaled_by_calcium.png" height="96%" width="96%">
<img src="files/media/issues_with_flow/flow_scaled_by_relative_calcium.png" height="96%" width="96%">

So I see that the scaling via the regular calcium seems to "sharpen" the flow within the processes and capture activity in the soma, while scaling via the relative fluorescence broadens the flow around the processes and silences the flow in the soma. Given our objective of measuring "flow" in the processes, off hand I think calculating the flow from the regular data and scaling it with the regular data produces the best results. Or perhaps I need to choose a better f0 to calculate the relative fluorescnece. What do you think?